In [1]:
import cv2
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from ultralytics import YOLO
import pandas as pd

# Load the pretrained model
model = YOLO("yolov8n.pt")

def send_email(image):
    # Set up the email server
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login("your_email@gmail.com", "your_password")  # Replace with your credentials

    # Create email message
    msg = MIMEMultipart()
    msg['From'] = "your_email@gmail.com"
    msg['To'] = "receiver_email@gmail.com"
    msg['Subject'] = "Intruder Alert"
    text = MIMEText("An intruder was detected. See the attached image.")
    msg.attach(text)

    # Attach the image
    img_data = cv2.imencode('.jpg', image)[1].tobytes()
    image = MIMEImage(img_data, name='intruder.jpg')
    msg.attach(image)

    # Send the email
    server.send_message(msg)
    server.quit()
    print("Email sent with image!")

100%|██████████| 6.23M/6.23M [00:00<00:00, 58.0MB/s]


In [2]:
def get_person_coordinates(frame):
    """Extracts coordinates of 'person' from model predictions."""
    results = model(frame)
    person_boxes = []
    if results:
        for result in results:
            boxes = result.boxes.data.detach().cpu().numpy()
            # Retrieve the index for 'person' from names dictionary
            person_index = [k for k, v in result.names.items() if v == 'person'][0]  # Assuming 'person' is a key
            # Filter out boxes detected as 'person'
            for box in boxes:
                if int(box[5]) == person_index and box[4] > 0.5:
                    person_boxes.append([box[0], box[1], box[2], box[3]])
    return person_boxes

def webcam_detect():
    cap = cv2.VideoCapture("http://192.168.1.17/live")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        person_coordinates = get_person_coordinates(frame)
        if person_coordinates:
            print("detection")
            #send_email(frame)  # Send the frame where the person was detected

        cv2.imshow('YOLOv8 Inference', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
webcam_detect()

[tcp @ 0x167bdd710] Connection to tcp://192.168.1.17:80 failed: Operation timed out
OpenCV: Couldn't read video stream from file "http://192.168.1.17/live"
